# Pre-processing script 2
This script pulls SDG data from API and transforms it into csv files.
The steps are:
- pivot into "wide" format

### Load necessary libraries

In [1]:
import csv
import json
import urllib3  # allows to access a URL with python
import math
import os
import io
import collections
import hashlib
import numpy as np
import pandas as pd
import re
import xlsxwriter

# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


### User parameters

In [2]:
release = '2019.Q1.G.03' # Make sure to have the correct release here

dir_path = os.path.dirname(os.path.realpath('__file__'))
print(dir_path)

wd_dir = r'../../'
print('data inputs dir: ' + wd_dir)

C:\Users\L.GonzalezMorales\Documents\GitHub\FIS4SDGs\notebooks\unsdPublishing
data inputs dir: ../../


## Utilities

#### Convert string to camelCase

In [3]:
def camelCase(st):
    """
    https://stackoverflow.com/questions/8347048/camelcase-every-string-any-standard-library
    
    """
    output = ''.join(x for x in st.title() if x.isalnum())
    return output[0].lower() + output[1:]

#### Disable insecure request warnings when using `urllib3`.

In [4]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#### Regular expression to capture numeric values (including those in scientific notation)
The regex is

```
-?      # an optional -
\d+     # a series of digits
(?:     # start non capturing group
  \.    # a dot
  \d+   # a series of digits
)?      
(?:     # start non capturing group
  e     # "e"
  -?    # an optional -
  \d+   # digits
)?
```


In [5]:
def numeric_part(v):
    numeric_part_f = re.compile(r'-?\d+(?:\.\d+)?(?:e-?\d+)?')
    x = numeric_part_f.findall(v)
    if len(x) > 0:
        return float(x[0])
    else:
        return None


#### Compute a hash of a dictionary

In [6]:
def dict_hash(d):
    out = hashlib.md5()
    for key, value in d.items():
        out.update(key.encode('utf-8'))
        out.update(str(value).encode('utf-8'))
    return out.hexdigest()


#### Get unique dictionaries in a list

In [7]:
def unique_dicts(dictionary_list):

    uniques_map = {}

    for d in dictionary_list:
        uniques_map[dict_hash(d)] = d

    return list(uniques_map.values())


#### Extract subset of key-value pairs from Python dictionary object

In [8]:
def subdict_list(dict_list, keys_list, exclude = False):
    sub_d_list = []
    if exclude:
        for d in dict_list:
            sub_d= {k: d[k] for k in d.keys() if k not in keys_list}
            sub_d_list.append(sub_d)
    else:
        for d in dict_list:
            if set(keys_list) <= set(d.keys()):
                sub_d= {k: d[k] for k in keys_list}
                sub_d_list.append(sub_d)
    
    return sub_d_list




#### Get a dict from a list based on something inside the dict

In [9]:
def select_dict(dict_list, k, v):
    selected = []
    for d in dict_list:
        if k in set(d.keys()):
            if d[k] == v:
                selected.append(d)
    return selected

#### Find the coverage of an (unordered) list of years

In [10]:
def year_intervals (years_list):
    """ Find the coverage of an ordered list of years"""
    
    years_list = list(map(int, years_list))
    
    years_list.sort()
    
    n = len(years_list)
    
    start_y = list()
    end_y = list()
    
    start_y.append(years_list[0])
    
    if n > 1:
        for i in range(n-1):
            if(years_list[i+1] - years_list[i]>1):
                start_y.append(years_list[i+1])
                end_y.append(years_list[i])
    
    end_y.append(years_list[n-1])
    
    interval_yy = list()
    
    for i in range(len(start_y)):

        if  end_y[i] - start_y[i]> 0 :
            interval_yy.append(str(start_y[i]) + '-' + str(end_y[i]))
        else:
            interval_yy.append(str(start_y[i]))

    
    x = ",".join(interval_yy)
    return(x)


In [11]:
year_intervals(['1995','2000', '1996', '2001','2002','2003','2004'])
year_intervals(['1995'])
year_intervals(['2000','2004'])

'1995-1996,2000-2004'

'1995'

'2000,2004'

### Read data file

In [ ]:
indicator = '10.6.1'
series = 'SG_INT_VRTDEV'


file = 'Indicator_'+indicator+'_Series_'+series+'.json'

with open(wd_dir + 'data/unsd/2019.Q1.G.03/' + file) as json_file:  
    data = json.load(json_file)

### Select `refAreas` that have coordinates

In [ ]:
ref_areas_publish = []
for d in data['refAreas']:
    if d['X'] and d['Y']:
        ref_areas_publish.append(d)


### Select distinct years among all data records:

In [ ]:
temp = []
timePeriods = []

for ra in ref_areas_publish:
    temp.extend(subdict_list(ra['data'], ['timePeriod'], exclude = False))
    
for i in unique_dicts(temp):
    timePeriods.append(i['timePeriod'])

timePeriods.sort()
timePeriods


### Select distinct slices among all data records

In [ ]:
temp = []

for ra in ref_areas_publish:
    temp.extend(subdict_list(ra['data'], ['timePeriod','timeDetail',
                                          'source','footnotes',
                                          'natureCode', 'natureDesc',
                                          'unitsCode', 'unitsDesc',
                                          'unitmultiplierCode', 'unitmultiplierDesc',
                                          'reportingTypeCode', 'reportingTypeDesc',
                                          'value_numeric_part',
                                          'value_is_censored',
                                          'value_detail'
                                         ], exclude = True))
    
slices = unique_dicts(temp)

In [ ]:
slices

### Write dataset as "pivot" version

In [39]:
with open(wd_dir + 'globalResources/metadata.json') as json_file:  
    metadata = json.load(json_file)


for g in metadata:
    for t in g['targets']:
        for i in t['indicators']:
            if 'series' in i.keys():
                for s in i['series']:
                    
                    # Read data file:
                    
                    file = 'Indicator_'+i['reference']+'_Series_'+s['code']+'.json'

                    with open(wd_dir + 'data/unsd/2019.Q1.G.03/' + file) as json_file:  
                        data = json.load(json_file)
                        
                    # Select only refAreas that have coordinates:
                        
                    ref_areas_publish = []
                    for d in data['refAreas']:
                        if d['X'] and d['Y']:
                            ref_areas_publish.append(d)
                            
                    # Select unique time periods among all records:
                            
                    temp = []
                    timePeriods = []

                    for ra in ref_areas_publish:
                        temp.extend(subdict_list(ra['data'], ['timePeriod'], exclude = False))

                    for dt in unique_dicts(temp):
                        timePeriods.append(dt['timePeriod'])

                    timePeriods.sort()
                    
                    temp = []
                    
                    # Obtain unique slices:

                    for ra in ref_areas_publish:
                        temp.extend(subdict_list(ra['data'], ['timePeriod','timeDetail',
                                                              'source','footnotes',
                                                              'natureCode', 'natureDesc',
                                                              'unitsCode', 'unitsDesc',
                                                              'unitmultiplierCode', 'unitmultiplierDesc',
                                                              'reportingTypeCode', 'reportingTypeDesc',
                                                              'value_numeric_part',
                                                              'value_is_censored',
                                                              'value_detail'
                                                             ], exclude = True))

                    slices = unique_dicts(temp)
                            
                    for ra in ref_areas_publish:
                        
                        data_new = []

                        for j in slices:
                            
                            # Select data corresponding to reference area ra and slice j:
                            slice_values = set(j.values())
                            slice_keys = set(j.keys())
                            slice_data = []

                            slice_data_wide = j.copy()

                            for record in ra['data']:
                                record_values = set(record.values())
                                if slice_values <= record_values:
                                    slice_data.append(record)

                            #----------------------------
                            slice_footnotes = []
                            slice_sources = []
                            slice_timeDetails = []
                            slice_years = []
                            slice_unitsCode = []
                            slice_unitsDesc = []
                            slice_unitmultiplierCode = []
                            slice_unitmultiplierDesc = []
                            slice_reportingTypeCode = []
                            slice_reportingTypeDesc = []


                            for r in slice_data:
                                if 'footnotes' in r.keys():
                                    slice_footnotes.append(r['footnotes'])
                                if 'source' in r.keys():
                                    slice_sources.append(r['source'])
                                if 'timeDetail' in r.keys():
                                    slice_timeDetails.append(r['timeDetail'])
                                if 'timePeriod' in r.keys():
                                    slice_years.append(r['timePeriod'])
                                if 'unitsCode' in r.keys():
                                    slice_unitsCode.append(r['unitsCode'])
                                if 'unitsDesc' in r.keys():
                                    slice_unitsDesc.append(r['unitsDesc'])
                                if 'unitmultiplierCode' in r.keys():
                                    slice_unitmultiplierCode.append(r['unitmultiplierCode'])
                                if 'unitmultiplierDesc' in r.keys():
                                    slice_unitmultiplierDesc.append(r['unitmultiplierDesc'])
                                if 'reportingTypeCode' in r.keys():
                                    slice_reportingTypeCode.append(r['reportingTypeCode'])
                                if 'reportingTypeDesc' in r.keys():
                                    slice_reportingTypeDesc.append(r['reportingTypeDesc'])

                            slice_footnotes = list(set(slice_footnotes))
                            slice_sources = list(set(slice_sources))
                            slice_timeDetails = list(set(slice_timeDetails))

                            max_year = None
                            if len(slice_years)>0:
                                max_year = max(slice_years)

                            slice_unitsCode = ' // '.join(list(set(slice_unitsCode)))
                            slice_unitsDesc = ' // '.join(list(set(slice_unitsDesc)))
                            slice_unitmultiplierCode = ' // '.join(list(set(slice_unitmultiplierCode)))
                            slice_unitmultiplierDesc = ' // '.join(list(set(slice_unitmultiplierDesc)))
                            slice_reportingTypeCode = ' // '.join(list(set(slice_reportingTypeCode)))
                            slice_reportingTypeDesc = ' // '.join(list(set(slice_reportingTypeDesc)))


                            slice_data_wide['unitsCode'] = slice_unitsCode
                            slice_data_wide['unitsDesc'] = slice_unitsDesc
                            slice_data_wide['unitmultiplierCode'] = slice_unitmultiplierCode
                            slice_data_wide['unitmultiplierDesc'] = slice_unitmultiplierDesc
                            slice_data_wide['reportingTypeCode'] = slice_reportingTypeCode
                            slice_data_wide['reportingTypeDesc'] = slice_reportingTypeDesc


                            #----------------------------

                            slice_footnote_join = []
                            counter = 0
                            for fn in slice_footnotes:
                                counter += 1
                                fn_years = []
                                for r in slice_data:
                                    if 'footnotes' in r.keys():
                                        if fn == r['footnotes']:
                                            fn_years.append(r['timePeriod'])
                                if fn:
                                    slice_footnote_join.append('['+year_intervals(fn_years)+']: ' + fn)
                                    
                                    

                            slice_footnote_join.sort()
                            slice_footnote_join = ' // '.join(slice_footnote_join)

                            slice_data_wide['footnotes'] = slice_footnote_join

                            #-----------------------

                            slice_sources_join = []
                            counter = 0
                            for src in slice_sources:
                                counter += 1
                                src_years = []
                                for r in slice_data:
                                    if src == r['source']:
                                        src_years.append(r['timePeriod'])
                                if src:
                                    slice_sources_join.append('['+year_intervals(src_years)+']: ' + src)

                            slice_sources_join.sort()
                            slice_sources_join = ' // '.join(slice_sources_join)

                            slice_data_wide['sources'] = slice_sources_join

                            #------------------------

                            slice_timeDetail_join = []
                            counter = 0
                            for td in slice_timeDetails:
                                counter += 1
                                td_years = []
                                for r in slice_data:
                                    if td == r['timeDetail']:
                                        td_years.append(r['timePeriod'])
                                if td:
                                    slice_timeDetail_join.append('['+year_intervals(td_years)+']: ' + td)

                            slice_timeDetail_join.sort()
                            slice_timeDetail_join = ' // '.join(slice_timeDetail_join)

                            slice_data_wide['timeDetails'] = slice_timeDetail_join

                            #------------------------

                            s_keys = list(slice_keys)
                            s_keys.extend(['footnotes','source', 'timeDetail',
                                           'unitsCode','unitsDesc',
                                           'unitmultiplierCode', 'unitmultiplierDesc',
                                           'reportingTypeCode','reportingTypeDesc'])


                            for y in timePeriods:

                                slice_data_y = subdict_list(select_dict(slice_data, 'timePeriod', y), 
                                                            s_keys, exclude = True)
                                if len(slice_data_y)>0:
                                    for ry in slice_data_y:
                                        if 'natureCode' in ry.keys():
                                            slice_data_wide['nature_'+str(y)] = ry['natureCode'] + ' : ' + ry['natureDesc']
                                        else:
                                            slice_data_wide['nature_'+str(y)] = None

                                        if 'value_numeric_part' in ry.keys():
                                            slice_data_wide['value_numeric_part_'+str(y)] = ry['value_numeric_part']
                                        else:
                                            slice_data_wide['value_numeric_part_'+str(y)] = None

                                        if 'value_is_censored' in ry.keys():
                                            slice_data_wide['value_is_censored_'+str(y)] = ry['value_is_censored'] 
                                        else:
                                            slice_data_wide['value_is_censored_'+str(y)] = None

                                        if 'value_detail' in ry.keys():
                                            slice_data_wide['value_detail_'+str(y)] = ry['value_detail'] 
                                        else:
                                            slice_data_wide['value_detail_'+str(y)] = None
                                else:

                                    slice_data_wide['nature_'+str(y)] = None
                                    slice_data_wide['value_numeric_part_'+str(y)] = None
                                    slice_data_wide['value_is_censored_'+str(y)] = None
                                    slice_data_wide['value_detail_'+str(y)] = None

                            if max_year:
                                slice_data_wide['value_numeric_part_latest'] = slice_data_wide['value_numeric_part_'+str(max_year)]
                                slice_data_wide['value_is_censored_latest'] = slice_data_wide['value_is_censored_'+str(max_year)]
                                slice_data_wide['value_detail_latest'] = slice_data_wide['value_detail_'+str(max_year)]
                                slice_data_wide['latest_year'] = max_year
                            else:
                                slice_data_wide['value_numeric_part_latest'] = None
                                slice_data_wide['value_is_censored_latest'] = None
                                slice_data_wide['value_detail_latest'] = None
                                slice_data_wide['latest_year'] = None



                            data_new.append(slice_data_wide)

                        del ra['data']
                        ra['data'] = data_new
                    
                    new_data = {}
                    new_data['goal'] = data['goal']
                    new_data['target'] = data['target']
                    new_data['indicator'] = data['indicator']
                    new_data['seriesCode'] = data['seriesCode']
                    new_data['seriesDesc'] = data['seriesDesc']
                    new_data['release'] = data['release']
                    new_data['data'] = ref_areas_publish
                    
                    
                    file_name = 'wide_Indicator_'+i['reference']+'_Series_'+s['code']+'.json'

                    with open(wd_dir + r'data\unsd\2019.Q1.G.03\\' + file_name, 'w') as f:
                        json.dump(new_data, f, indent=4)
                        
                    
                    print(file_name)

                    

Indicator_1.1.1_Series_SI_POV_DAY1.json
1.1.1 - SI_POV_DAY1


[1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017]

----
wide_Indicator_1.1.1_Series_SI_POV_DAY1.json
Indicator_1.1.1_Series_SI_POV_EMP1.json
1.1.1 - SI_POV_EMP1


[2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017]

----
wide_Indicator_1.1.1_Series_SI_POV_EMP1.json
Indicator_1.2.1_Series_SI_POV_NAHC.json
1.2.1 - SI_POV_NAHC


...

----
wide_Indicator_17.19.2_Series_SG_REG_CENSUSN.json
Indicator_17.19.2_Series_SG_REG_BRTH90N.json
17.19.2 - SG_REG_BRTH90N


[2015, 2016]

----
wide_Indicator_17.19.2_Series_SG_REG_BRTH90N.json
Indicator_17.19.2_Series_SG_REG_DETH75N.json
17.19.2 - SG_REG_DETH75N


[2015, 2016]

----
wide_Indicator_17.19.2_Series_SG_REG_DETH75N.json
